In [1]:
import kfp
import tfx
import os

### Upload the dataset

In [3]:
DATA_ROOT_URI='gs://acc-demo-artifact-store/lab-datasets/online_news'

!gsutil cp gs://workshop-datasets/online_news/full/data.csv $DATA_ROOT_URI/data.csv 

Copying gs://workshop-datasets/online_news/full/data.csv [Content-Type=text/csv]...
/ [1 files][ 20.1 MiB/ 20.1 MiB]                                                
Operation completed over 1 objects/20.1 MiB.                                     


### Build the trainer image

In [6]:
PROJECT_ID='mlops-demo'
IMAGE_NAME='popularity-predictor-trainer'
TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

!gcloud builds submit --timeout 15m --tag $IMAGE_URI . 

Creating temporary tarball archive of 8 file(s) totalling 29.8 KiB before compression.
Uploading tarball of [.] to [gs://mlops-demo_cloudbuild/source/1578626288.07-7ba8c9cfb54d4130a987ab34e11623ab.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/mlops-demo/builds/44d474cc-6302-45ea-9e80-8bbc05ca5201].
Logs are available at [https://console.cloud.google.com/gcr/builds/44d474cc-6302-45ea-9e80-8bbc05ca5201?project=609934025272].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "44d474cc-6302-45ea-9e80-8bbc05ca5201"

FETCHSOURCE
Fetching storage object: gs://mlops-demo_cloudbuild/source/1578626288.07-7ba8c9cfb54d4130a987ab34e11623ab.tgz#1578626288641563
Copying gs://mlops-demo_cloudbuild/source/1578626288.07-7ba8c9cfb54d4130a987ab34e11623ab.tgz#1578626288641563...
/ [1 files][  7.0 KiB/  7.0 KiB]                                                
Operation completed over 1 objects/7.0 KiB.                                      
BUILD
Al

### Compile the pipeline

In [8]:
%%sh

export PROJECT_ID=mlops-demo
export ARTIFACT_STORE_URI=gs://mlops-demo-artifact-store
export DATA_ROOT_URI=gs://mlops-demo-artifact-store/lab-datasets/online_news
export TFX_IMAGE=gcr.io/mlops-demo/popularity-predictor-trainer:latest
export PIPELINE_NAME=popularity_predictor_training
export GCP_REGION=us-central1
export RUNTIME_VERSION=1.15
export PYTHON_VERSION=3.7


tfx pipeline compile --pipeline_path pipeline_dsl.py 

CLI
Compiling pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Pipeline compiled successfully.
Pipeline package path: /home/mlops-demo/tfx-pipeline/pipeline-dsl/popularity_predictor_training.tar.gz


In [19]:
%env PROJECT_ID=mlops-demo
%env ARTIFACT_STORE_URI=gs://mlops-demo-artifact-store
%env DATA_ROOT_URI=gs://mlops-demo-artifact-store/lab-datasets/online_news
%env TFX_IMAGE=gcr.io/mlops-demo/popularity-predictor-trainer:latest
%env PIPELINE_NAME=popularity_predictor_training
%env PACKAGE_PATH=popularity_predictor_training.tar.gz
%env GCP_REGION=us-central1
%env RUNTIME_VERSION=1.15
%env PYTHON_VERSION=3.7

!tfx pipeline compile --pipeline_path pipeline_dsl.py --package_path $PACKAGE_PATH

env: PROJECT_ID=mlops-demo
env: ARTIFACT_STORE_URI=gs://mlops-demo-artifact-store
env: DATA_ROOT_URI=gs://mlops-demo-artifact-store/lab-datasets/online_news
env: TFX_IMAGE=gcr.io/mlops-demo/popularity-predictor-trainer:latest
env: PIPELINE_NAME=popularity_predictor_training
env: PACKAGE_PATH=popularity_predictor_training.tar.gz
env: GCP_REGION=us-central1
env: RUNTIME_VERSION=1.15
env: PYTHON_VERSION=3.7
CLI
Compiling pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Pipeline compiled successfully.
Pipeline package path: popularity_predictor_training.tar.gz


In [15]:
%env

{'ARTIFACT_STORE_URI': 'gs://mlops-demo-artifact-store',
 'CLICOLOR': '1',
 'CONDA_DEFAULT_ENV': 'base',
 'CONDA_EXE': '/root/miniconda3/bin/conda',
 'CONDA_PREFIX': '/root/miniconda3',
 'CONDA_PROMPT_MODIFIER': '(base) ',
 'CONDA_PYTHON_EXE': '/root/miniconda3/bin/python',
 'CONDA_ROOT': '/root/miniconda3',
 'CONDA_SHLVL': '1',
 'DATA_ROOT_URI': 'gs://mlops-demo-artifact-store/lab-datasets/online_news',
 'GCE_METADATA_TIMEOUT': '3',
 'GCP_REGION': 'us-central1',
 'GIT_PAGER': 'cat',
 'GIT_PYTHON_REFRESH': 'quiet',
 'HOME': '/root',
 'HOSTNAME': 'f0b0c4694a51',
 'JPY_PARENT_PID': '7',
 'LANG': 'C.UTF-8',
 'LC_ALL': 'C.UTF-8',
 'MPLBACKEND': 'module://ipykernel.pylab.backend_inline',
 'NO_GCE_CHECK': 'False',
 'PAGER': 'cat',
 'PATH': '/root/miniconda3/bin:/root/miniconda3/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
 'PIPELINE_NAME': 'popularity_predictor_training',
 'PROJECT_ID': 'mlops-demo',
 'PWD': '/',
 'PYTHON_VERSION': '3.7',
 'RUNTIME_VERSION': '1.15',
 'S

### Upload the pipeline

In [21]:
%env INVERSE_PROXY_HOST=1ff32660e53f5d89-dot-us-central1.notebooks.googleusercontent.com

!kfp --endpoint $INVERSE_PROXY_HOST pipeline upload -p $PIPELINE_NAME $PACKAGE_PATH

env: INVERSE_PROXY_HOST=1ff32660e53f5d89-dot-us-central1.notebooks.googleusercontent.com
Pipeline 19cfbd88-5239-4a8b-943e-ebad802a9a22 has been submitted

Pipeline Details
------------------
ID           19cfbd88-5239-4a8b-943e-ebad802a9a22
Name         popularity_predictor_training
Description
Uploaded at  2020-01-10T03:36:45+00:00
+------------------+--------------------------------------------------------------+
| Parameter Name   | Default Value                                                |
+==================+==============================================================+
| pipeline-root    | gs://mlops-demo-artifact-store/popularity_predictor_training |
+------------------+--------------------------------------------------------------+


### List pipelines

In [22]:
!kfp --endpoint $INVERSE_PROXY_HOST pipeline list

+--------------------------------------+----------------------------------------------------------+---------------------------+
| Pipeline ID                          | Name                                                     | Uploaded at               |
+======================================+==========================================================+===========================+
| 19cfbd88-5239-4a8b-943e-ebad802a9a22 | popularity_predictor_training                            | 2020-01-10T03:36:45+00:00 |
+--------------------------------------+----------------------------------------------------------+---------------------------+
| 7a589200-0397-4e72-815f-66c3fc78b08e | covertype_classifier_training_R.0.1                      | 2020-01-10T03:05:20+00:00 |
+--------------------------------------+----------------------------------------------------------+---------------------------+
| f1246bd6-cef1-4476-b8a1-74e0457b63f2 | covertype_classifier_training_test                       | 2020

### Submit the run

In [23]:
%env EXPERIMENT_NAME=Popularity_Predictor_Training
%env RUN_NAME=Training_Run_1
%env PIPELINE_ID 19cfbd88-5239-4a8b-943e-ebad802a9a22

!kfp --endpoint $INVERSE_PROXY_HOST run submit \
-e $EXPERIMENT_NAME \
-r RUN_NAME \
-p $PIPELINE_ID


env: EXPERIMENT_NAME=Popularity_Predictor_Training
env: RUN_NAME=Training_Run_1
env: PIPELINE_ID=19cfbd88-5239-4a8b-943e-ebad802a9a22
Creating experiment Popularity_Predictor_Training.
Run 5ef3884a-3492-47c2-aaa3-919dd13aa560 is submitted
+--------------------------------------+----------+----------+---------------------------+
| run id                               | name     | status   | created at                |
+======================================+==========+==========+===========================+
| 5ef3884a-3492-47c2-aaa3-919dd13aa560 | RUN_NAME |          | 2020-01-10T03:40:57+00:00 |
+--------------------------------------+----------+----------+---------------------------+
